In [1]:
# import module

import time
import boto3
import datetime
import cx_Oracle
import pandas as pd
import os
os.environ['LD_LIBRARY_PATH']=':/ORACLE/db/12c/lib'

In [2]:
# Create an SNS client
client = boto3.client(
    "sns",
    aws_access_key_id="",
    aws_secret_access_key="",
    region_name=""
)

In [3]:
# DB connencting

con = cx_Oracle.connect('fimsr/vudrk_read@192.168.1.130:1521/FIMS2005')
con2 = cx_Oracle.connect('11834/3793@192.168.1.127:1521/FUNDDB')
con3 = cx_Oracle.connect('11834/3793@192.168.1.151:1521/IDX01')

cur_fims2005 = con.cursor()
cur_funddb = con2.cursor()
cur_idx01 = con3.cursor()

In [4]:
# PRINT SQL Result

tablespace_query='SELECT TBS_NM \
                                    , tbs_sz                                as "Total(MB)"\
                                    , tbs_alloc_sz                          as "Alloc(MB)"\
                                    , tbs_alloc_sz - tbs_free_sz            as "Used(MB)"\
                                    , round((tbs_alloc_sz-tbs_free_sz)/tbs_sz*100,1) AS "Used(%)"\
                                    , tbs_sz - (tbs_alloc_sz - tbs_free_sz) as "Free(MB)"\
                                    FROM\
                                    (\
                                            SELECT tbs_nm\
                                            , max(tbs_alloc_sz) as tbs_alloc_sz\
                                            , max(tbs_sz)       as tbs_sz\
                                            , max(tbs_free_sz)        as tbs_free_sz\
                                            FROM\
                                            (\
                                                    select tablespace_name                      as tbs_nm\
                                                    , round(sum(bytes)/power(1024,2),2)    as tbs_alloc_sz\
                                                    , round(sum(MAXBYTES)/power(1024,2),2) as tbs_sz\
                                                    , 0                                    As tbs_free_sz\
                                                    from dba_data_files\
                                                    group by tablespace_name\
                                                    union all\
                                                    select distinct\
                                                    tablespace_name as tbs_nm\
                                                    , 0               as tbs_alloc_sz\
                                                    , 0               as tbs_sz\
                                                    , round(sum(bytes) over (partition by tablespace_name)/power(1024,2)) As tbs_free_sz\
                                                    from dba_free_space\
                                                    )\
                                            group by tbs_nm\
                                            ) where 0=0 order by 5 desc'

In [5]:
# Input Oracle tablespace_amount_used to Pandas DataFrame

fims2005_tablespace  = pd.read_sql(tablespace_query,con,index_col=None)
funddb_tablespace = pd.read_sql(tablespace_query,con2,index_col=None)
idx01_tablespace  = pd.read_sql(tablespace_query,con3,index_col=None)
fims2005_tablespace.head()

,TBS_NM,Total(MB),Alloc(MB),Used(MB),Used(%),Free(MB)
0,TFNDATA_JT2016,100000,99001,95037,95.0,4963
1,TPFDATA_JT2013,90000,86002,84972,94.4,5028
2,TPFINDX_BF_2012,170000,164002,160514,94.4,9486
3,TSTINDX_02,100000,95000,93610,93.6,6390
4,TPFDATA_BF_2012,180000,170001,166043,92.2,13957


In [6]:
# add DB_NAME to DataFrame
fims2005_tablespace['DB_NAME']='FIMS2005'
funddb_tablespace['DB_NAME']='FUNDDB'
idx01_tablespace['DB_NAME']='IDX01'

# concat each tablespace_DataFrame
all_tablespace = pd.concat([fims2005_tablespace, funddb_tablespace, idx01_tablespace]).reset_index(drop=True)
all_tablespace.head()

,TBS_NM,Total(MB),Alloc(MB),Used(MB),Used(%),Free(MB),DB_NAME
0,TFNDATA_JT2016,100000,99001.0,95037.0,95.0,4963.0,FIMS2005
1,TPFDATA_JT2013,90000,86002.0,84972.0,94.4,5028.0,FIMS2005
2,TPFINDX_BF_2012,170000,164002.0,160514.0,94.4,9486.0,FIMS2005
3,TSTINDX_02,100000,95000.0,93610.0,93.6,6390.0,FIMS2005
4,TPFDATA_BF_2012,180000,170001.0,166043.0,92.2,13957.0,FIMS2005


In [7]:
# if amount_used > 95% record Data to list
all_tablespace_result = []
for i in range(len(all_tablespace)):
    if all_tablespace['Used(%)'][i]>94:
        all_tablespace_result.append(all_tablespace.iloc[i])
all_tablespace_result

[TBS_NM       TFNDATA_JT2016
 Total(MB)            100000
 Alloc(MB)             99001
 Used(MB)              95037
 Used(%)                  95
 Free(MB)               4963
 DB_NAME            FIMS2005
 Name: 0, dtype: object, TBS_NM       TPFDATA_JT2013
 Total(MB)             90000
 Alloc(MB)             86002
 Used(MB)              84972
 Used(%)                94.4
 Free(MB)               5028
 DB_NAME            FIMS2005
 Name: 1, dtype: object, TBS_NM       TPFINDX_BF_2012
 Total(MB)             170000
 Alloc(MB)             164002
 Used(MB)              160514
 Used(%)                 94.4
 Free(MB)                9486
 DB_NAME             FIMS2005
 Name: 2, dtype: object]

In [8]:
# formatting to text
all_tablespace_result2 = []
if len(all_tablespace_result)>=1:
    for i in range(len(all_tablespace_result)):
        all_tablespace_result2.append('{}. {}의 {} 사용률이 {}% 입니다'.format('{}'.format(i+1),all_tablespace_result[i]['DB_NAME'],
                                      all_tablespace_result[i]['TBS_NM'],all_tablespace_result[i]['Used(%)']))
all_tablespace_result3 = []
all_tablespace_result3 = '\n'.join(str(x) for x in all_tablespace_result2)

all_tablespace_result3

'1. FIMS2005의 TFNDATA_JT2016 사용률이 95.0% 입니다\n2. FIMS2005의 TPFDATA_JT2013 사용률이 94.4% 입니다\n3. FIMS2005의 TPFINDX_BF_2012 사용률이 94.4% 입니다'

In [9]:
# print SQL Result 2
datafile_query="select TABLESPACE_NAME\
                    , FILE_ID\
                    , FILE_NAME\
                    , round(sum(bytes)/power(1024,2),2) AS MB\
                    , AUTOEXTENSIBLE\
                    , round(sum(MAXBYTES)/power(1024,2),2) AS MAXBYTES\
                   from DBA_DATA_FILES\
                   where tablespace_name= '{}'\
                   group by TABLESPACE_NAME, FILE_ID, FILE_NAME, AUTOEXTENSIBLE\
                   order by 2"

datafile_query2=[]
if all_tablespace_result:
    for i in range(len(all_tablespace_result)):
        datafile_query2.append(datafile_query.format(all_tablespace_result[i]['TBS_NM']))

datafile_query2[0]

"select TABLESPACE_NAME                    , FILE_ID                    , FILE_NAME                    , round(sum(bytes)/power(1024,2),2) AS MB                    , AUTOEXTENSIBLE                    , round(sum(MAXBYTES)/power(1024,2),2) AS MAXBYTES                   from DBA_DATA_FILES                   where tablespace_name= 'TFNDATA_JT2016'                   group by TABLESPACE_NAME, FILE_ID, FILE_NAME, AUTOEXTENSIBLE                   order by 2"

In [10]:
# Input Oracle Data_file to Pandas DataFrame
datafile_fims2005=[]
datafile_funddb=[]
datafile_idx01=[]
if datafile_query2:
    for i in range(len(datafile_query2)):
        if all_tablespace_result[i]['DB_NAME']=='FIMS2005':
            datafile_fims2005.append(pd.read_sql(datafile_query2[i],con,index_col=None))
        elif all_tablespace_result[i]['DB_NAME']=='FUNDDB':
            datafile_funddb.append(pd.read_sql(datafile_query2[i],con2,index_col=None))
        else:
            datafile_idx01.append(pd.read_sql(datafile_query2[i],con3,index_col=None))

datafile_all = datafile_fims2005 + datafile_funddb + datafile_idx01
datafile_all[0]

,TABLESPACE_NAME,FILE_ID,FILE_NAME,MB,AUTOEXTENSIBLE,MAXBYTES
0,TFNDATA_JT2016,18,/ORADATA01/DATA/TFN/TFNDATA_JT2016_01.DBF,10000,YES,10000
1,TFNDATA_JT2016,135,/ORADATA02/DATA/TFN/TFNDATA_JT2016_02.DBF,10000,YES,10000
2,TFNDATA_JT2016,258,/ORADATA03/DATA/TFN/TFNDATA_JT2016_03.DBF,10000,YES,10000
3,TFNDATA_JT2016,323,/ORADATA04/DATA/TFN/TFNDATA_JT2016_04.DBF,10000,YES,10000
4,TFNDATA_JT2016,326,/ORADATA05/DATA/TFN/TFNDATA_JT2016_05.DBF,9001,YES,10000
5,TFNDATA_JT2016,327,/ORADATA06/DATA/TFN/TFNDATA_JT2016_06.DBF,10000,YES,10000
6,TFNDATA_JT2016,328,/ORADATA07/DATA/TFN/TFNDATA_JT2016_07.DBF,10000,YES,10000
7,TFNDATA_JT2016,329,/ORADATA08/DATA/TFN/TFNDATA_JT2016_08.DBF,10000,YES,10000
8,TFNDATA_JT2016,330,/ORADATA09/DATA/TFN/TFNDATA_JT2016_09.DBF,10000,YES,10000
9,TFNDATA_JT2016,350,/ORADATA10/DATA/TFN/TFNDATA_JT2016_10.DBF,10000,YES,10000


In [11]:
# from DataFrame to formatting text
datafile_all2=[]
if datafile_all:
    for i in range(len(datafile_all)):
        data=[]
        for j in range(len(datafile_all[i])):
            if datafile_all[i].iloc[j]['TABLESPACE_NAME'] == all_tablespace_result[i]['TBS_NM']:
                data.append('\n'.join(str(x) for x in list(datafile_all[i].iloc[j])).replace('\n',' '))
        datafile_all2.append(data)

datafile_all3=[]
if datafile_all2:
    for i in range(len(datafile_all)):
        datafile_all3.append('{}. {}'.format(i+1,all_tablespace_result[i]['TBS_NM']) + '\n'
                             + '\n'.join(str(x) for x in list(datafile_all[i].columns)).replace('\n',' ') +'\n'
                             + '\n'.join(str(x) for x in list(datafile_all2[i])))
datafile_all3_2= '\n'.join(str(x) for x in datafile_all3)
datafile_all3_2

'1. TFNDATA_JT2016\nTABLESPACE_NAME FILE_ID FILE_NAME MB AUTOEXTENSIBLE MAXBYTES\nTFNDATA_JT2016 18 /ORADATA01/DATA/TFN/TFNDATA_JT2016_01.DBF 10000 YES 10000\nTFNDATA_JT2016 135 /ORADATA02/DATA/TFN/TFNDATA_JT2016_02.DBF 10000 YES 10000\nTFNDATA_JT2016 258 /ORADATA03/DATA/TFN/TFNDATA_JT2016_03.DBF 10000 YES 10000\nTFNDATA_JT2016 323 /ORADATA04/DATA/TFN/TFNDATA_JT2016_04.DBF 10000 YES 10000\nTFNDATA_JT2016 326 /ORADATA05/DATA/TFN/TFNDATA_JT2016_05.DBF 9001 YES 10000\nTFNDATA_JT2016 327 /ORADATA06/DATA/TFN/TFNDATA_JT2016_06.DBF 10000 YES 10000\nTFNDATA_JT2016 328 /ORADATA07/DATA/TFN/TFNDATA_JT2016_07.DBF 10000 YES 10000\nTFNDATA_JT2016 329 /ORADATA08/DATA/TFN/TFNDATA_JT2016_08.DBF 10000 YES 10000\nTFNDATA_JT2016 330 /ORADATA09/DATA/TFN/TFNDATA_JT2016_09.DBF 10000 YES 10000\nTFNDATA_JT2016 350 /ORADATA10/DATA/TFN/TFNDATA_JT2016_10.DBF 10000 YES 10000\n2. TPFDATA_JT2013\nTABLESPACE_NAME FILE_ID FILE_NAME MB AUTOEXTENSIBLE MAXBYTES\nTPFDATA_JT2013 141 /ORADATA02/DATA/TPF/TPFDATA_JT2013_01.DB

In [12]:
# classify Scale up or Scale out and show sql_query
new=[]
if datafile_all:
    for i in range(len(datafile_all)):
        if int(datafile_all[i]['FILE_NAME'][len(datafile_all[i])-1][8:10]) == 10:
            new.append(datafile_all[i]['FILE_NAME'][len(datafile_all[i])-1].replace(datafile_all[i]['FILE_NAME'][len(datafile_all[i])-1][8:10],
                       str('01'), 1).replace(datafile_all[i]['FILE_NAME'][len(datafile_all[i])-1][-6:-4], str(int(datafile_all[i]['FILE_NAME'][len(datafile_all[i])-1][-6:-4])+1)))
        else:
            new.append(datafile_all[i]['FILE_NAME'][len(datafile_all[i])-1].replace(datafile_all[i]['FILE_NAME'][len(datafile_all[i])-1][8:10], 
            str(int(datafile_all[i]['FILE_NAME'][len(datafile_all[i])-1][8:10])+1), 1).replace(datafile_all[i]['FILE_NAME'][len(datafile_all[i])-1][-6:-4],
            str(int(datafile_all[i]['FILE_NAME'][len(datafile_all[i])-1][-6:-4])+1)))

scale_up = []
scale_out = []
if datafile_all:
    for i in range(len(datafile_all)):
        for j in range(len(datafile_all[i])):
            if datafile_all[i]['MAXBYTES'][j] != 10000:
                scale_up.append("{}. ALTER DATABASE DATAFILE '{}' AUTOETEND ON NEXT 1000M MAXSIZE 10000M".format(i+1, datafile_all[i]['FILE_NAME'][j]))
        if len(datafile_all[i][datafile_all[i].MAXBYTES != 10000]) == 0:
            scale_out.append("{}. ALTER TABLESPACE {} ADD DATAFILE '{}' SIZE 1M AUTOEXTEND ON NEXT 1000M MAXSIZE 10000M".format(i+1, all_tablespace_result[i]['TBS_NM'],new[i]))

scale_up2 = '\n'.join(str(x) for x in scale_up)
scale_out2 = '\n'.join(str(x) for x in scale_out)
scale_out2

"1. ALTER TABLESPACE TFNDATA_JT2016 ADD DATAFILE '/ORADATA01/DATA/TFN/TFNDATA_JT2016_11.DBF' SIZE 1M AUTOEXTEND ON NEXT 1000M MAXSIZE 10000M\n2. ALTER TABLESPACE TPFDATA_JT2013 ADD DATAFILE '/ORADATA01/DATA/TPF/TPFDATA_JT2013_10.DBF' SIZE 1M AUTOEXTEND ON NEXT 1000M MAXSIZE 10000M\n3. ALTER TABLESPACE TPFINDX_BF_2012 ADD DATAFILE '/ORADATA10/INDX/TPF/TPFINDX_BF_2012_18.DBF' SIZE 1M AUTOEXTEND ON NEXT 1000M MAXSIZE 10000M"

In [13]:
# combine all formatting text
if all_tablespace_result3:    
    sns = all_tablespace_result3 + '\n\n' + datafile_all3_2 + '\n\n - Scale up query- \n' + scale_up2 + '\n\n - Scale out query- \n' + scale_out2

sns

"1. FIMS2005의 TFNDATA_JT2016 사용률이 95.0% 입니다\n2. FIMS2005의 TPFDATA_JT2013 사용률이 94.4% 입니다\n3. FIMS2005의 TPFINDX_BF_2012 사용률이 94.4% 입니다\n\n1. TFNDATA_JT2016\nTABLESPACE_NAME FILE_ID FILE_NAME MB AUTOEXTENSIBLE MAXBYTES\nTFNDATA_JT2016 18 /ORADATA01/DATA/TFN/TFNDATA_JT2016_01.DBF 10000 YES 10000\nTFNDATA_JT2016 135 /ORADATA02/DATA/TFN/TFNDATA_JT2016_02.DBF 10000 YES 10000\nTFNDATA_JT2016 258 /ORADATA03/DATA/TFN/TFNDATA_JT2016_03.DBF 10000 YES 10000\nTFNDATA_JT2016 323 /ORADATA04/DATA/TFN/TFNDATA_JT2016_04.DBF 10000 YES 10000\nTFNDATA_JT2016 326 /ORADATA05/DATA/TFN/TFNDATA_JT2016_05.DBF 9001 YES 10000\nTFNDATA_JT2016 327 /ORADATA06/DATA/TFN/TFNDATA_JT2016_06.DBF 10000 YES 10000\nTFNDATA_JT2016 328 /ORADATA07/DATA/TFN/TFNDATA_JT2016_07.DBF 10000 YES 10000\nTFNDATA_JT2016 329 /ORADATA08/DATA/TFN/TFNDATA_JT2016_08.DBF 10000 YES 10000\nTFNDATA_JT2016 330 /ORADATA09/DATA/TFN/TFNDATA_JT2016_09.DBF 10000 YES 10000\nTFNDATA_JT2016 350 /ORADATA10/DATA/TFN/TFNDATA_JT2016_10.DBF 10000 YES 10000\n2. TP

In [15]:
if sns:
    print(sns)

1. FIMS2005의 TFNDATA_JT2016 사용률이 95.0% 입니다
2. FIMS2005의 TPFDATA_JT2013 사용률이 94.4% 입니다
3. FIMS2005의 TPFINDX_BF_2012 사용률이 94.4% 입니다

1. TFNDATA_JT2016
TABLESPACE_NAME FILE_ID FILE_NAME MB AUTOEXTENSIBLE MAXBYTES
TFNDATA_JT2016 18 /ORADATA01/DATA/TFN/TFNDATA_JT2016_01.DBF 10000 YES 10000
TFNDATA_JT2016 135 /ORADATA02/DATA/TFN/TFNDATA_JT2016_02.DBF 10000 YES 10000
TFNDATA_JT2016 258 /ORADATA03/DATA/TFN/TFNDATA_JT2016_03.DBF 10000 YES 10000
TFNDATA_JT2016 323 /ORADATA04/DATA/TFN/TFNDATA_JT2016_04.DBF 10000 YES 10000
TFNDATA_JT2016 326 /ORADATA05/DATA/TFN/TFNDATA_JT2016_05.DBF 9001 YES 10000
TFNDATA_JT2016 327 /ORADATA06/DATA/TFN/TFNDATA_JT2016_06.DBF 10000 YES 10000
TFNDATA_JT2016 328 /ORADATA07/DATA/TFN/TFNDATA_JT2016_07.DBF 10000 YES 10000
TFNDATA_JT2016 329 /ORADATA08/DATA/TFN/TFNDATA_JT2016_08.DBF 10000 YES 10000
TFNDATA_JT2016 330 /ORADATA09/DATA/TFN/TFNDATA_JT2016_09.DBF 10000 YES 10000
TFNDATA_JT2016 350 /ORADATA10/DATA/TFN/TFNDATA_JT2016_10.DBF 10000 YES 10000
2. TPFDATA_JT2013
TABL

In [16]:
# DB Closing
cur_fims2005.close()
cur_funddb.close()
cur_idx01.close()

con.close()
con2.close()
con3.close()